## Johdanto

Tässä oppitunnissa käsitellään:
- Mitä funktiokutsu on ja sen käyttötapaukset
- Kuinka luoda funktiokutsu OpenAI:n avulla
- Kuinka integroida funktiokutsu sovellukseen

## Oppimistavoitteet

Oppitunnin suorittamisen jälkeen osaat ja ymmärrät:

- Funktiokutsun käytön tarkoituksen
- Funktiokutsun määrittämisen OpenAI-palvelussa
- Kuinka suunnitella tehokkaita funktiokutsuja sovelluksesi käyttötarkoitukseen


## Funktiokutsujen ymmärtäminen

Tässä oppitunnissa haluamme rakentaa ominaisuuden koulutusstartupillemme, joka antaa käyttäjien käyttää chatbotia teknisten kurssien löytämiseen. Suosittelemme kursseja, jotka sopivat heidän taitotasoonsa, nykyiseen rooliinsa ja kiinnostuksen kohteena olevaan teknologiaan.

Tämän toteuttamiseksi käytämme yhdistelmää:
 - `OpenAI` luomaan käyttäjälle chat-kokemuksen
 - `Microsoft Learn Catalog API` auttamaan käyttäjiä löytämään kursseja käyttäjän pyynnön perusteella
 - `Funktiokutsua` ottamaan käyttäjän kyselyn ja lähettämään se funktiolle API-pyynnön tekemiseksi.

Aloittaaksemme katsotaan, miksi haluaisimme käyttää funktiokutsua ylipäätään:

print("Viestit seuraavassa pyynnössä:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # saa uuden vastauksen GPT:ltä, jossa se voi nähdä funktion vastauksen


print(second_response.choices[0].message)


### Miksi funktiokutsu

Jos olet suorittanut minkä tahansa muun oppitunnin tässä kurssissa, ymmärrät todennäköisesti suurten kielimallien (LLM) käytön voiman. Toivottavasti näet myös joitakin niiden rajoituksia.

Funktiokutsu on OpenAI-palvelun ominaisuus, joka on suunniteltu ratkaisemaan seuraavat haasteet:

Epäjohdonmukainen vastausten muotoilu:
- Ennen funktiokutsua suurten kielimallien vastaukset olivat jäsentämättömiä ja epäjohdonmukaisia. Kehittäjien piti kirjoittaa monimutkaista validointikoodia käsitelläkseen jokaisen tulosvaihtelun.

Rajoitettu integraatio ulkoisten tietojen kanssa:
- Ennen tätä ominaisuutta oli vaikeaa sisällyttää sovelluksen muiden osien tietoja keskustelukontekstiin.

Vakioimalla vastausmuodot ja mahdollistamalla saumaton integraatio ulkoisten tietojen kanssa funktiokutsu yksinkertaistaa kehitystä ja vähentää lisävalidointilogiikan tarvetta.

Käyttäjät eivät voineet saada vastauksia kuten "Mikä on tämänhetkinen sää Tukholmassa?". Tämä johtuu siitä, että mallit olivat rajoitettuja siihen aikaan, jolloin data oli koulutettu.

Katsotaan alla olevaa esimerkkiä, joka havainnollistaa tätä ongelmaa:

Oletetaan, että haluamme luoda opiskelijatietokannan, jotta voimme ehdottaa heille oikeaa kurssia. Alla on kaksi kuvausta opiskelijoista, jotka ovat hyvin samankaltaisia sisältämiensä tietojen osalta.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Haluamme lähettää tämän LLM:lle tietojen jäsentämistä varten. Tätä voidaan myöhemmin käyttää sovelluksessamme lähettämään tiedot API:lle tai tallentamaan ne tietokantaan.

Luodaan kaksi identtistä kehotetta, joissa ohjeistamme LLM:ää siitä, mistä tiedoista olemme kiinnostuneita:


Haluamme lähettää tämän LLM:lle jäsentämään osat, jotka ovat tärkeitä tuotteellemme. Joten voimme luoda kaksi identtistä kehotetta ohjaamaan LLM:ää:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Näiden kahden kehotteen luomisen jälkeen lähetämme ne LLM:lle käyttämällä `openai.ChatCompletion`-toimintoa. Tallennamme kehotteen `messages`-muuttujaan ja asetamme rooliksi `user`. Tämä jäljittelee käyttäjän kirjoittamaa viestiä chatbotille.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Nyt voimme lähettää molemmat pyynnöt LLM:lle ja tarkastella saamaamme vastausta.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Vaikka kehotteet ovat samat ja kuvaukset samankaltaisia, voimme saada erilaisia muotoja `Grades`-ominaisuudelle.

Jos suoritat yllä olevan solun useita kertoja, muoto voi olla `3.7` tai `3.7 GPA`.

Tämä johtuu siitä, että LLM ottaa vastaan jäsentämätöntä dataa kirjoitetun kehotteen muodossa ja palauttaa myös jäsentämätöntä dataa. Tarvitsemme jäsennellyn muodon, jotta tiedämme, mitä odottaa tallennettaessa tai käytettäessä tätä dataa.

Käyttämällä funktion kutsumista voimme varmistaa, että saamme takaisin jäsenneltyä dataa. Funktion kutsumisen yhteydessä LLM ei itse asiassa kutsu tai suorita mitään funktioita. Sen sijaan luomme rakenteen, jota LLM noudattaa vastauksissaan. Käytämme sitten näitä jäsenneltyjä vastauksia tietääksemme, mitä funktiota sovelluksissamme suoritetaan.


![Funktiokutsun kulku kaavio](../../../../translated_images/fi/Function-Flow.083875364af4f4bb.webp)


Voimme sitten ottaa funktion palauttaman arvon ja lähettää sen takaisin LLM:lle. LLM vastaa luonnollisella kielellä käyttäjän kysymykseen.


### Käyttötapaukset funktiokutsujen käyttämiseen

**Ulkoisten työkalujen kutsuminen**  
Chatbotit ovat erinomaisia vastaamaan käyttäjien kysymyksiin. Funktiokutsujen avulla chatbotit voivat käyttää käyttäjien viestejä tiettyjen tehtävien suorittamiseen. Esimerkiksi opiskelija voi pyytää chatbotilta "Lähetä sähköposti ohjaajalleni, jossa sanon tarvitsevani lisää apua tässä aiheessa". Tämä voi tehdä funktiokutsun `send_email(to: string, body: string)`

**API- tai tietokantakyselyiden luominen**  
Käyttäjät voivat löytää tietoa luonnollisella kielellä, joka muunnetaan muotoilluksi kyselyksi tai API-pyynnöksi. Esimerkkinä voisi olla opettaja, joka kysyy "Ketkä opiskelijat ovat suorittaneet viimeisen tehtävän", mikä voisi kutsua funktiota nimeltä `get_completed(student_name: string, assignment: int, current_status: string)`

**Rakenteellisen datan luominen**  
Käyttäjät voivat ottaa tekstilohkon tai CSV-tiedoston ja käyttää LLM:ää tärkeän tiedon poimimiseen siitä. Esimerkiksi opiskelija voi muuntaa Wikipedia-artikkelin rauhansopimuksista luodakseen tekoälymuistikortteja. Tämä voidaan tehdä käyttämällä funktiota nimeltä `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Ensimmäisen funktiokutsun luominen

Funktiokutsun luomiseen kuuluu 3 päävaihetta:
1. Chat Completions -rajapinnan kutsuminen funktiolistallasi ja käyttäjän viestillä
2. Mallin vastauksen lukeminen toiminnon suorittamiseksi, eli funktion tai API-kutsun suorittaminen
3. Toinen kutsu Chat Completions -rajapintaan funktion vastauksella, jotta tätä tietoa voidaan käyttää vastauksen luomiseen käyttäjälle.


![Funktion kutsun kulku](../../../../translated_images/fi/LLM-Flow.3285ed8caf4796d7.webp)


### Funktion kutsun osat

#### Käyttäjän syöte

Ensimmäinen vaihe on luoda käyttäjän viesti. Tämä voidaan määrittää dynaamisesti ottamalla arvo tekstisyötteestä tai voit määrittää arvon tässä. Jos käytät Chat Completions API:a ensimmäistä kertaa, meidän täytyy määritellä viestin `role` ja `content`.

`role` voi olla joko `system` (sääntöjen luominen), `assistant` (malli) tai `user` (loppukäyttäjä). Funktiokutsujen yhteydessä määritämme tämän arvoksi `user` ja annamme esimerkkikysymyksen.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Funktioiden luominen.

Seuraavaksi määrittelemme funktion ja sen parametrit. Käytämme tässä vain yhtä funktiota nimeltä `search_courses`, mutta voit luoda useita funktioita.

**Tärkeää**: Funktiot sisällytetään järjestelmäviestiin LLM:lle, ja ne lasketaan käytettävissä olevien tokenien määrään.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Määritelmät** 

Funktiomääritelmän rakenne sisältää useita tasoja, joista jokaisella on omat ominaisuutensa. Tässä on erittely sisäkkäisestä rakenteesta:

**Ylimmän tason funktion ominaisuudet:**

`name` - Funktion nimi, jota haluamme kutsuttavan. 

`description` - Kuvaus siitä, miten funktio toimii. Tässä on tärkeää olla tarkka ja selkeä. 

`parameters` - Lista arvoista ja muodosta, jonka haluat mallin tuottavan vastauksessaan. 

**Parametrien objektin ominaisuudet:**

`type` - Parametrien objektin tietotyyppi (yleensä "object")

`properties` - Lista erityisistä arvoista, joita malli käyttää vastauksessaan. 

**Yksittäisten parametrien ominaisuudet:**

`name` - Määritellään implisiittisesti ominaisuuden avaimella (esim. "role", "product", "level")

`type` - Tämän tietyn parametrin tietotyyppi (esim. "string", "number", "boolean") 

`description` - Kuvaus tästä erityisestä parametrista. 

**Valinnaiset ominaisuudet:**

`required` - Taulukko, joka listaa, mitkä parametrit ovat pakollisia funktion kutsun suorittamiseksi. 


### Funktion kutsuminen  
Kun funktio on määritelty, meidän täytyy nyt sisällyttää se Chat Completion API -kutsuun. Teemme tämän lisäämällä `functions` pyyntöön. Tässä tapauksessa `functions=functions`.  

On myös vaihtoehto asettaa `function_call` arvoksi `auto`. Tämä tarkoittaa, että annamme LLM:n päättää, mikä funktio tulisi kutsua käyttäjän viestin perusteella sen sijaan, että määrittäisimme sen itse.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Katsotaanpa nyt vastausta ja miten se on muotoiltu:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Voit nähdä, että funktion nimi kutsutaan ja käyttäjän viestin perusteella LLM pystyi löytämään tiedot, jotka sopivat funktion argumentteihin.


## 3. Funktiokutsujen integroiminen sovellukseen. 


Kun olemme testanneet muotoillun vastauksen LLM:ltä, voimme nyt integroida tämän sovellukseen. 

### Virran hallinta 

Integroidaksemme tämän sovellukseemme, tehdään seuraavat vaiheet: 

Ensiksi tehdään kutsu OpenAI-palveluihin ja tallennetaan viesti muuttujaan nimeltä `response_message`. 


In [ ]:
response_message = response.choices[0].message

Nyt määrittelemme funktion, joka kutsuu Microsoft Learn -rajapintaa saadakseen kurssilistan:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Parhaana käytäntönä tarkistamme sitten, haluaako malli kutsua funktiota. Tämän jälkeen luomme yhden käytettävissä olevista funktioista ja vastaamme sen funktioon, jota kutsutaan.  
Otamme sitten funktion argumentit ja yhdistämme ne LLM:n argumentteihin.

Lopuksi liitämme funktiokutsun viestin ja arvot, jotka palautettiin `search_courses`-viestillä. Tämä antaa LLM:lle kaiken tarvittavan tiedon vastatakseen käyttäjälle luonnollisella kielellä.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Nyt lähetämme päivitetyn viestin LLM:lle, jotta voimme saada luonnollisen kielen vastauksen API:n JSON-muotoisen vastauksen sijaan.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Koodhaushaaste

Hienoa työtä! Jatkaaksesi OpenAI Function Calling -oppimista voit rakentaa: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - Lisää funktiolle parametreja, jotka voivat auttaa oppijoita löytämään enemmän kursseja. Saatavilla olevat API-parametrit löytyvät täältä:  
 - Luo toinen funktiokutsu, joka ottaa oppijalta enemmän tietoja, kuten heidän äidinkielensä  
 - Luo virheenkäsittely, kun funktiokutsu ja/tai API-kutsu ei palauta sopivia kursseja  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Vastuuvapauslauseke**:
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, otathan huomioon, että automaattikäännöksissä saattaa esiintyä virheitä tai epätarkkuuksia. Alkuperäinen asiakirja sen alkuperäiskielellä on virallinen lähde. Tärkeissä asioissa suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa tämän käännöksen käytöstä aiheutuvista väärinymmärryksistä tai tulkinnoista.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
